# temp

In [1]:
import torch

split_file="/data/targetdiff_data/sample_output/PDBBind_filtered_by_DUD_E_BLAST_13537/result_1.pt"
data=torch.load(split_file)



/home/tanhaichuan/.conda/envs/sbdd/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data

dict_keys(['data', 'pred_ligand_pos', 'pred_ligand_v', 'pred_ligand_pos_traj', 'pred_ligand_v_traj', 'time'])

In [ ]:
data['data']

In [2]:
tmp_file="/data/rag/data_splits/PDBBind_filtered_by_DUD_E_FLAPP_0.5.pkl"
import pickle
with open(tmp_file, "rb") as f:
    index=pickle.load(f)
print(index)

{'train': ['1mfd', '3sgx', '1lcj', '2lsp', '4bj8', '3uil', '3dda', '5alw', '1ym1', '2llq', '6qto', '5lz4', '2cc7', '3zw3', '2m3o', '2qrk', '6fx1', '4o7d', '4ozj', '4oyp', '5v2q', '6bmx', '4v25', '5huw', '5lyh', '4fxf', '2vhj', '2yim', '5n55', '1vcu', '5o1c', '6dq4', '1e4h', '6b1k', '5yc3', '4wkn', '3vw1', '3t0w', '5ai8', '6hq3', '6q9o', '5jgi', '4hp0', '3srb', '5hk9', '6abp', '1txr', '3atw', '4jr5', '5vqe', '4ycw', '2c9t', '4yv9', '6md8', '6dim', '5l8o', '4pkw', '1sqb', '4yih', '4dfw', '3zha', '3rl3', '5epy', '4rww', '3zjt', '4xkb', '5u48', '5n69', '5fl0', '4g0y', '6gj7', '5fsy', '5j18', '4ju4', '6nzv', '3jzi', '5ive', '4yqm', '6ft4', '5acw', '2kgi', '3rwg', '5knx', '6f08', '4hs8', '3lp4', '3uvl', '5vfc', '5t2y', '3vbg', '4cp5', '3eju', '4uuh', '2w8w', '1iy7', '5od1', '3uxm', '1i7c', '1hn2', '5zyi', '4ef4', '6bxy', '4yas', '4jt9', '3tyv', '2yne', '5olb', '3pn1', '5fov', '6aaq', '4zx9', '1zd3', '3meu', '2qta', '5ngs', '6j10', '5epp', '2gfk', '6fgc', '5dde', '6fs1', '2kfh', '6ccm', '1pot

In [2]:
num_data=10
key=f'{num_data:08d}'.encode()
print(key)

b'00000010'


In [1]:
import lmdb
import os
from tqdm import tqdm
import pickle
def read_lmdb(lmdb_path, mode="idx"):
    """
    Read lmdb file.

    Args:
        lmdb_path (str): Path to the lmdb file.
        mode (str, optional): Read mode. "idx" to follow the idx order, "direct" to read the data directly (use when idx is not continuous).

    Returns:
        list: List of data read from the lmdb file.
    """
    env = lmdb.open(
        lmdb_path,
        subdir=False,
        readonly=True,
        lock=False,
        readahead=False,
        meminit=False,
        max_readers=256,
    )
    pocket_name_cnt = {}
    smi_cnt={}
    pocket_smi_pair_cnt={}
    txn = env.begin()
    keys = list(txn.cursor().iternext(values=False))
    data_all = []
    if mode == "idx":
        for idx in tqdm(range(len(keys)), desc="read lmdb {}".format(lmdb_path)):
            ky=f'{idx}'.encode()
            datapoint_pickled = txn.get(ky)
            data_piece = pickle.loads(datapoint_pickled)
            data_all.append(data_piece)
            smi=data_piece['smi']
            pocket_name=data_piece['pocket_name']
            pocket_smi_pair=(smi, pocket_name)
            smi_cnt[smi]=smi_cnt.get(smi, 0)+1
            pocket_name_cnt[pocket_name]=pocket_name_cnt.get(pocket_name, 0)+1
            pocket_smi_pair_cnt[pocket_smi_pair]=pocket_smi_pair_cnt.get(pocket_smi_pair, 0)+1
    elif mode == "direct":
        for key in tqdm(keys, desc="read lmdb {}".format(lmdb_path)):
            datapoint_pickled = txn.get(key)
            data_piece = pickle.loads(datapoint_pickled)
            data_all.append(data_piece)
    return data_all, smi_cnt, pocket_name_cnt, pocket_smi_pair_cnt

lmdb_path="/data/BioLip/AR_utils/BioLip.lmdb"
data,*_=read_lmdb(lmdb_path, mode="direct")

read lmdb /data/BioLip/AR_utils/BioLip.lmdb:   0%|          | 0/50180 [00:00<?, ?it/s]

read lmdb /data/BioLip/AR_utils/BioLip.lmdb: 100%|██████████| 50180/50180 [00:44<00:00, 1121.92it/s]


In [3]:
data[0].keys()

dict_keys(['protein_element', 'protein_pos', 'protein_is_backbone', 'protein_atom_name', 'protein_atom_to_aa_type', 'ligand_smiles', 'ligand_element', 'ligand_pos', 'ligand_bond_index', 'ligand_bond_type', 'ligand_center_of_mass', 'ligand_atom_feature', 'ligand_hybridization', 'ligand_nbh_list', 'protein_filename', 'ligand_filename'])

In [2]:
data[0]['ligand_bond_index']

tensor([[ 0,  1,  1,  1,  2,  2,  2,  3,  3,  3,  4,  5,  6,  6,  7,  7,  7,  8,
          8,  8,  9,  9, 10, 10, 11, 11, 11, 12, 12, 12, 12, 13, 14, 15, 16, 16,
         17, 17, 18, 18],
        [ 1,  0,  2, 18,  1,  3,  6,  2,  4,  5,  3,  3,  2,  7,  6,  8, 18,  7,
          9, 17,  8, 10,  9, 11, 10, 12, 16, 11, 13, 14, 15, 12, 12, 12, 11, 17,
          8, 16,  1,  7]])

In [4]:
elements_cnt={}
for d in tqdm(data):
    elements=d['ligand_element']
    for e in elements:
        e=e.item()
        elements_cnt[e]=elements_cnt.get(e, 0)+1
elements_cnt
elements_cnt.keys()

100%|██████████| 19379/19379 [00:01<00:00, 18454.97it/s]


dict_keys([6, 7, 16, 8, 9, 35, 15, 17, 53, 5, 4, 14, 44, 26, 33, 34, 77, 29, 1, 23, 51, 75, 12, 78, 30])

In [16]:
tmp_file="/data/rag/data_splits/PDBBind_filtered_by_DUD_E_FLAPP_0.5.pkl"
import pickle
import rdkit
from rdkit import Chem
from tqdm import tqdm 
with open(tmp_file, "rb") as f:
    index=pickle.load(f)
print(index)
# random sample 1000


import sys 
import os
sys.path.append("/project/ProFSA")
from scripts.benchmark.dataset import PDBBindDataset, DUDEDataset ,BioLipDataset
dataset=PDBBindDataset()
smiles_list=[]
for item in tqdm(dataset):
    name=item['name']
    try:
        mol=Chem.MolFromMol2File(item['ligand_dir'])
        smi=Chem.MolToSmiles(mol)
        smiles_list.append(smi)
    except KeyboardInterrupt:
        raise KeyboardInterrupt
    except:
        print(name)
        continue


{'train': ['1mfd', '3sgx', '1lcj', '2lsp', '4bj8', '3uil', '3dda', '5alw', '1ym1', '2llq', '6qto', '5lz4', '2cc7', '3zw3', '2m3o', '2qrk', '6fx1', '4o7d', '4ozj', '4oyp', '5v2q', '6bmx', '4v25', '5huw', '5lyh', '4fxf', '2vhj', '2yim', '5n55', '1vcu', '5o1c', '6dq4', '1e4h', '6b1k', '5yc3', '4wkn', '3vw1', '3t0w', '5ai8', '6hq3', '6q9o', '5jgi', '4hp0', '3srb', '5hk9', '6abp', '1txr', '3atw', '4jr5', '5vqe', '4ycw', '2c9t', '4yv9', '6md8', '6dim', '5l8o', '4pkw', '1sqb', '4yih', '4dfw', '3zha', '3rl3', '5epy', '4rww', '3zjt', '4xkb', '5u48', '5n69', '5fl0', '4g0y', '6gj7', '5fsy', '5j18', '4ju4', '6nzv', '3jzi', '5ive', '4yqm', '6ft4', '5acw', '2kgi', '3rwg', '5knx', '6f08', '4hs8', '3lp4', '3uvl', '5vfc', '5t2y', '3vbg', '4cp5', '3eju', '4uuh', '2w8w', '1iy7', '5od1', '3uxm', '1i7c', '1hn2', '5zyi', '4ef4', '6bxy', '4yas', '4jt9', '3tyv', '2yne', '5olb', '3pn1', '5fov', '6aaq', '4zx9', '1zd3', '3meu', '2qta', '5ngs', '6j10', '5epp', '2gfk', '6fgc', '5dde', '6fs1', '2kfh', '6ccm', '1pot

32it [00:00, 300.24it/s][17:16:45] Can't kekulize mol.  Unkekulized atoms: 2 4 6 7 8 9 10 11 12
119it [00:00, 381.22it/s]6:45] 4kw6_ligand: 

4kw6


258it [00:00, 409.13it/s][17:16:46] Explicit valence for atom # 2 O, 3, is greater than permitted
342it [00:00, 399.32it/s]6:46] 3bho_ligand_experimental: 

3bho
5yjy


...

17802it [00:47, 410.92it/s][17:17:33] Can't kekulize mol.  Unkekulized atoms: 0 1 3 4 5
[17:17:33] sanitize [17:17:33] 1sl3_ligand: [17:17:33] Explicit valence for atom # 10 C, 5, is greater than permitted
17889it [00:47, 406.61it/s][17:17:33] sanitize [17:17:33] 1h07_ligand: 

1sl3
1h07


17937it [00:47, 426.48it/s][17:17:33] 6mny_ligand: Warning - no explicit hydrogens in mol2 file but needed for formal charge estimation.
18080it [00:48, 449.54it/s][17:17:34] 1ao8_ligand: warning - charged amidine and isFixed atom.
[17:17:34] 4ew2_ligand: warning - charged amidine and isFixed atom.
18583it [00:49, 420.28it/s][17:17:35] Can't kekulize mol.  Unkekulized atoms: 31 32 33 34 35
18682it [00:49, 444.36it/s]35] 2fm5_ligand: 

2fm5


18865it [00:50, 421.26it/s][17:17:35] 5jid_ligand: Warning - no explicit hydrogens in mol2 file but needed for formal charge estimation.
19438it [00:51, 377.78it/s]


In [20]:
import random
random.seed(0)
smiles_1000_list=random.sample(smiles_list, 1000)
# save in a file
tmp_file="/data/rag/PDBBind_random_mol_1k.txt"
with open(tmp_file, "w") as f:
    for smi in smiles_1000_list:
        f.write(smi+"\n")

In [5]:
import os
a="/data"
b="biolip/test"
os.path.join(a, b)

'/data/biolip//test'

In [5]:
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
file="/data/BioLip/raw/4mjq_27R_A_401/ligand.sdf"
mol=Chem.MolFromMolFile(file,sanitize=False)
# print bond type
#sanitization
bond_typs_old=[]
for bond in mol.GetBonds():
    bond_typs_old.append(bond.GetBondType())


Chem.SanitizeMol(mol)


#kekulize
# Chem.Kekulize(mol, clearAromaticFlags=True)
bond_typs_new=[]
for bond in mol.GetBonds():
    bond_typs_new.append(bond.GetBondType())
for old, new in zip(bond_typs_old, bond_typs_new):
    # if old!=new:
        print(old, new)

SINGLE AROMATIC
DOUBLE AROMATIC
DOUBLE AROMATIC
SINGLE AROMATIC
DOUBLE AROMATIC
DOUBLE DOUBLE
SINGLE SINGLE
SINGLE SINGLE
SINGLE SINGLE
DOUBLE AROMATIC
SINGLE AROMATIC
SINGLE AROMATIC
SINGLE SINGLE
SINGLE SINGLE
DOUBLE DOUBLE
SINGLE SINGLE
SINGLE AROMATIC
SINGLE SINGLE
SINGLE AROMATIC
DOUBLE AROMATIC
DOUBLE AROMATIC


In [5]:
from rdkit import Chem

mol = Chem.MolFromMolFile("/data/BioLip/raw/7q7i_9I8_A_4000/ligand.sdf")

frags = Chem.GetMolFrags(mol)
print(frags)
is_disconnected = len(frags) > 1

print(is_disconnected)  # 输出：True

[14:20:26] Explicit valence for atom # 5 C, 5, is greater than permitted


ArgumentError: Python argument types in
    rdkit.Chem.rdmolops.GetMolFrags(NoneType)
did not match C++ signature:
    GetMolFrags(RDKit::ROMol mol, bool asMols=False, bool sanitizeFrags=True, boost::python::api::object frags=None, boost::python::api::object fragsMolAtomMapping=None)

In [2]:
import pickle

file="/data/DUD-E/DUD-E.pkl"
with open(file, "rb") as f:
    index=pickle.load(f)
print(index)

[('hivpr/pocket10A.pdb', 'hivpr/crystal_ligand.mol2'), ('akt2/pocket10A.pdb', 'akt2/crystal_ligand.mol2'), ('abl1/pocket10A.pdb', 'abl1/crystal_ligand.mol2'), ('pparg/pocket10A.pdb', 'pparg/crystal_ligand.mol2'), ('dpp4/pocket10A.pdb', 'dpp4/crystal_ligand.mol2'), ('tgfr1/pocket10A.pdb', 'tgfr1/crystal_ligand.mol2'), ('gria2/pocket10A.pdb', 'gria2/crystal_ligand.mol2'), ('hxk4/pocket10A.pdb', 'hxk4/crystal_ligand.mol2'), ('fa7/pocket10A.pdb', 'fa7/crystal_ligand.mol2'), ('try1/pocket10A.pdb', 'try1/crystal_ligand.mol2'), ('mcr/pocket10A.pdb', 'mcr/crystal_ligand.mol2'), ('pygm/pocket10A.pdb', 'pygm/crystal_ligand.mol2'), ('fnta/pocket10A.pdb', 'fnta/crystal_ligand.mol2'), ('pgh1/pocket10A.pdb', 'pgh1/crystal_ligand.mol2'), ('mapk2/pocket10A.pdb', 'mapk2/crystal_ligand.mol2'), ('hs90a/pocket10A.pdb', 'hs90a/crystal_ligand.mol2'), ('csf1r/pocket10A.pdb', 'csf1r/crystal_ligand.mol2'), ('cp3a4/pocket10A.pdb', 'cp3a4/crystal_ligand.mol2'), ('cp2c9/pocket10A.pdb', 'cp2c9/crystal_ligand.mol2'